In [2]:
import glob
from langchain_community.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.schema import Document
import pymupdf
import os

In [3]:
# PDF dosyasını yükleme
pdf_folders_path =glob.glob("../knowledge_base/*")
text_loader_kwargs = {'encoding': 'utf-8'}
print(pdf_folders_path)


['../knowledge_base/3GPP', '../knowledge_base/articles']


In [4]:
chunked_documents = []

for pdf_folder in pdf_folders_path:
    
    print(f"Processing folder: {pdf_folder}")

    loader = DirectoryLoader(pdf_folder, glob="**/*.pdf", loader_cls=PyMuPDFLoader)
    folder_docs = loader.load()


    pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))


    for pdf_path in pdf_files:
        # PDF dosyasını aç
        pdf_open = pymupdf.open(pdf_path)
        
        # Table of Contents (TOC) al
        toc = pdf_open.get_toc()

        for i, item in enumerate(toc):
            heading = item[1]
            start_page = item[2]

            if i + 1 < len(toc):
                end_page = toc[i + 1][2] - 1
            else:
                end_page = pdf_open.page_count  # Son başlıksa PDF'in son sayfasına kadar
    
            # Sonraki başlığın sayfasına kadar olan kısmı almak
            if i + 1 < len(toc):
                end_page = toc[i + 1][2] - 1
            else:
                end_page = pdf_open.page_count -  1   # Son başlıksa PDF'in son sayfasına kadar

            # Metni birleştirerek chunk oluştur
            chunk_text = ""
            for page_num in range(start_page, end_page):  # PyMuPDF için 0-index
                chunk_text += pdf_open[page_num].get_text()

        # İlk chunk'ları oluştur ve listeye ekle
            chunked_documents.append(
                Document(
                    page_content=chunk_text,
                    metadata={"heading": heading, "start_page": start_page, "end_page": end_page}
                )
            )


Processing folder: ../knowledge_base/3GPP
Processing folder: ../knowledge_base/articles


In [5]:
# CharacterTextSplitter kullanarak ek parçalama
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# TOC'ye göre bölünmüş metinleri tekrar split ediyoruz
final_chunks = text_splitter.split_documents(chunked_documents)

# Çıktıyı kontrol etmek için

In [6]:
len(chunked_documents)

1072

In [7]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

# Create our Chroma vectorstore!
db_name = "vector-db"

In [8]:
import torch
from langchain_chroma import Chroma
import os

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create our Chroma vectorstore!

vectorstore = Chroma.from_documents(documents=final_chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 1416 documents
